RETRIEVING TORONTO POSTAL CODE DATA FROM WIKIPEDIA

In [2]:
from bs4 import BeautifulSoup
import requests as rq 
import pandas as pd
from pandas.compat import StringIO as io

In [3]:
#Get data in table from website

site = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
StoreSite = rq.get(site)
soup = BeautifulSoup(StoreSite.text, 'html.parser')
table = soup.find("table", class_="wikitable sortable")

In [9]:
#Turn data into a srting and conform it to a csv

data1 = table.text.replace('\n',',')
data2 = data1.replace(',,,','\n')
data3 = data2.replace(',,','')
data3

"Postcode,Borough,Neighbourhood\nM1A,Not assigned,Not assigned\nM2A,Not assigned,Not assigned\nM3A,North York,Parkwoods\nM4A,North York,Victoria Village\nM5A,Downtown Toronto,Harbourfront\nM5A,Downtown Toronto,Regent Park\nM6A,North York,Lawrence Heights\nM6A,North York,Lawrence Manor\nM7A,Queen's Park,Not assigned\nM8A,Not assigned,Not assigned\nM9A,Etobicoke,Islington Avenue\nM1B,Scarborough,Rouge\nM1B,Scarborough,Malvern\nM2B,Not assigned,Not assigned\nM3B,North York,Don Mills North\nM4B,East York,Woodbine Gardens\nM4B,East York,Parkview Hill\nM5B,Downtown Toronto,Ryerson\nM5B,Downtown Toronto,Garden District\nM6B,North York,Glencairn\nM7B,Not assigned,Not assigned\nM8B,Not assigned,Not assigned\nM9B,Etobicoke,Cloverdale\nM9B,Etobicoke,Islington\nM9B,Etobicoke,Martin Grove\nM9B,Etobicoke,Princess Gardens\nM9B,Etobicoke,West Deane Park\nM1C,Scarborough,Highland Creek\nM1C,Scarborough,Rouge Hill\nM1C,Scarborough,Port Union\nM2C,Not assigned,Not assigned\nM3C,North York,Flemingdon Park

In [10]:
#Create Canada postal codes (cpc) dataframe from string

cpc_df1 = pd.read_csv(io(data3))

cpc_df1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
#Drop rows with unassigned borough 
cpc_df2 = cpc_df1[cpc_df1['Borough']!='Not assigned'].reset_index(drop=True)

#Merge postal codes with multiple neighbourhoods
cpc_df3 = cpc_df2.groupby(['Postcode', 'Borough',])['Neighbourhood'].apply(', '.join).reset_index()

#Check how many boroughs without a neighbourhood 
cpc_df3_check = cpc_df3[cpc_df3['Neighbourhood']=='Not assigned']

#Assign bourough value to 'Not assigned' neighbourhood
cpc_df3.loc[85, 'Neighbourhood'] = cpc_df3.loc[85, 'Borough'] 

In [7]:
#Final cpc dataframe cleaned  
cpc_df = cpc_df3
cpc_df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
cpc_df.shape

(103, 3)